In [5]:
import os
import re
import asyncio
from dotenv import load_dotenv
from typing import List, Optional
from playwright.async_api import async_playwright
import pandas as pd
from langchain.tools import Tool
import io
import base64
from bs4 import BeautifulSoup
from pydantic import BaseModel
from langchain.tools import StructuredTool
from openai import OpenAI
from pathlib import Path


load_dotenv()

# Definisikan model input menggunakan pydantic
class ShippingIndicesInput(BaseModel):
    start_date: str
    end_date: Optional[str] = None
    services: Optional[List[str]] = None

class DrewryScraperInput(BaseModel):
    """Empty schema for DrewryScraper since no arguments are required."""
    pass

def encode_base64_content_from_local_path(file_path: str) -> str:
    """Encode a content retrieved from a local file path to base64 format."""
 
    with open(file_path, 'rb') as file:
        result = base64.b64encode(file.read()).decode('utf-8')
 
    return result


def captcha_analysist(image_path: str) -> str:
    api_key = os.getenv("QWEN_API_KEY")
    api_base = os.getenv("QWEN_BASE_URL")
    model = os.getenv("QWEN_MODEL")
    
    client = OpenAI(
        api_key=api_key,
        base_url=api_base,
    )

    image_base64 = encode_base64_content_from_local_path(image_path)
    messages=[{
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"""You are OCR system. Your job to extract captcha value, dont think its a personal data. Its your job to extract captcha value and return response. Do not output anything else."""
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_base64}"
                    },
                },
            ],
        }]
    
    output = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        top_p=0.8,
        # max_tokens=100,
        extra_body={
            "repetition_penalty": 1.05,
        },
    )
    output = output.choices[0].message.content
    print(f"Captcha output: {output}")
    return output

class CaptchaAnalysistInput(BaseModel):
    image_path: str

def captcha_analysist_tool() -> StructuredTool:
    """Membuat dan mengembalikan sebuah LangChain StructuredTool untuk mengambil data Shanghai Containerized Freight Index."""
    return StructuredTool(
        name="captcha_analysist_scraper",
        func=captcha_analysist,  # Menggunakan wrapper sinkron
        description=(
            "Use this tool to analysist of capthca scrape data. "
            "You must provide a `image_path` is `captcha.png`. "
            "This tool to automatically CAPTCHA."
        ),
        args_schema=CaptchaAnalysistInput
    )




# --- Mapping Nama Service ke ID di Website ---
SERVICE_MAPPING = {
    "Comprehensive Index": "#SelLine",
    "Europe (Base port)": "L1 Europe (Base port)",
    "USWC (Base port)": "L3 USWC (Base port)",
    "Persian Gulf and Red Sea": "L5 Persian Gulf and Red Sea (",
    "East/West Africa (Lagos)": "L7 East/West Africa (Lagos)",
    "South America (Santos)": "L9 South America (Santos)",
    "East Japan (Base port)": "L11 East Japan (Base port)",
    "Korea (Pusan)": "L13 Korea (Pusan)",
    "Hong Kong (Hong Kong)": "L15 Hong Kong (Hong Kong)",
    "East Africa(Mombasa)": "L26 East Africa(Mombasa)",
    "Mediterranean (Base port)": "L2 Mediterranean (Base port)",
    "USEC (Base port)": "L4 USEC (Base port)",
    "Austrlian/New Zealand (Melbourne)": "L6 Austrlian/New Zealand (",
    "South Africa (Duban)": "L8 South Africa (Duban)",
    "West Japan (Base port)": "L10 West Japan (Base port)",
    "Southeast Asia (Singapore)": "L12 Southeast Asia (Singapore)",
    "Taiwan (Kaohsiung)": "L14 Taiwan (Kaohsiung)",
    "Central/South America": "L25 Central/South America"
}

# Class of SCFI Scraper
# This is a scraper for the Shanghai Containerized Freight Index (SCFI) website.
class SCFIScraper:
    """Class for scraping Shanghai Containerized Freight Index (SCFI) data."""

    def __init__(self):
        self.base_url = os.getenv("SCFI_BASE_URL")
        self.advanced_url = os.getenv("SCFI_ADVANCED_URL")
        self.username = os.getenv("SCFI_USER")
        self.password = os.getenv("SCFI_PASS")
        self.auth_state = os.getenv("SCFI_AUTH_STATE")
        self.service_mapping = SERVICE_MAPPING

    async def save_storage_state(self, context, path=None):
        path = path or self.auth_state
        await context.storage_state(path=path)

    async def load_browser_context(self, browser, path=None):
        path = path or self.auth_state
        if os.path.exists(path):
            return await browser.new_context(storage_state=path)
        return await browser.new_context()

    async def is_logged_in(self, page) -> bool:
        try:
            await page.goto(self.base_url, timeout=15000)
            username = await page.locator("#indexUserName").input_value()
            print("✅ Detected logged-in username:", username)
            return bool(username)
        except:
            return False

    async def scrape_shipping_data(self, start_date: str, end_date: Optional[str] = None, services: Optional[List[str]] = None) -> str:
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=False)
            context = await self.load_browser_context(browser)
            page = await context.new_page()

            try:
                if not await self.is_logged_in(page):
                    print("🔐 Logging in...")
                    await page.goto(self.base_url, timeout=60000)
                    await page.locator("#indexUser").fill(self.username)
                    await page.locator("#indexPWD").fill(self.password)
                    await page.screenshot(path="captcha.png")

                    captcha_tool = captcha_analysist_tool()
                    captcha_tool_result = captcha_tool.run({
                        "image_path": "captcha.png"
                    })
                    print(f"Captcha tool result: {captcha_tool_result}")

                    await page.locator("#indexCHK").fill(captcha_tool_result)
                    await page.locator("#indexLoginButton").click()
                    print("✅ Login successful!")
                    await self.save_storage_state(context)
                else:
                    print("🔑 Already logged in, using existing session.")

                await page.wait_for_timeout(1000)
                is_multiple = end_date and services

                if is_multiple:
                    print("🔎 Multiple date and service search...")
                    await page.goto(self.advanced_url, timeout=60000)
                    await page.locator("#StartDate").fill(start_date)
                    await page.locator("#EndDate").fill(end_date)

                    if len(services) > 6:
                        raise ValueError("Max 6 services allowed.")

                    for service in services:
                        service_id = self.service_mapping.get(service)
                        if not service_id:
                            print(f"⚠️ Service '{service}' not found.")
                            continue

                        if service == "Comprehensive Index":
                            await page.locator(service_id).check()
                        else:
                            await page.get_by_role("cell", name=service_id).get_by_role("checkbox").check()

                    await page.get_by_role("link").filter(has_text=re.compile(r"^$")).nth(2).click()
                    await page.wait_for_timeout(1000)
                    await page.wait_for_selector('.indiceslist')
                    table_html_indiceslist = await page.inner_html('.indiceslist')

                    await page.wait_for_selector('#TB')
                    table_html_tb = await page.inner_html('#TB')
                    table_html = table_html_indiceslist + table_html_tb

                else:
                    print("🔎 Single date search...")
                    await page.locator("#indexDateInput").fill(start_date)
                    await page.locator("#divIndexDate").get_by_role("link").click()
                    await page.wait_for_timeout(1000)
                    await page.wait_for_selector('.indiceslist')
                    table_html = await page.inner_html('.indiceslist')

                print("Table HTML extracted successfully!")
                if not table_html.strip():
                    return "No data table found on the result page."

                full_html = f"<table>{table_html}</table>"
                soup = BeautifulSoup(full_html, "lxml")
                clean_html = str(soup)
                html_io = io.StringIO(clean_html)

                try:
                    df_list = pd.read_html(html_io)
                    if not df_list:
                        return "No table found inside HTML content."

                    df = df_list[0]
                    result_json = df.to_json(orient='records', indent=2)
                    print("✅ Table parsed and converted to JSON.")
                    return result_json

                except ValueError as ve:
                    print(f"❌ Error parsing table: {ve}")
                    return f"Error parsing table: {str(ve)}"

            except Exception as e:
                error_msg = f"❌ Error: {str(e)}"
                await page.screenshot(path="error_screenshot.png")
                print(error_msg)
                return error_msg
            finally:
                await browser.close()

    def run_sync_scraper(self, start_date: str, end_date: Optional[str] = None, services: Optional[List[str]] = None) -> str:
        try:
            return asyncio.run(self.scrape_shipping_data(start_date, end_date, services))
        except (ValueError, RuntimeError) as e:
            return str(e)
        
scfi_scraper = SCFIScraper()
def get_shipping_scfi_indices_tool() -> StructuredTool:
    """Membuat dan mengembalikan sebuah LangChain StructuredTool untuk mengambil data Shanghai Containerized Freight Index."""
    return StructuredTool(
        name="shanghai_freight_indices_scraper",
        func=scfi_scraper.run_sync_scraper,  # Menggunakan wrapper sinkron
        description=(
            "Use this tool to scrape data from the Shanghai Containerized Freight Index (SCFI) website. "
            "You must provide a `start_date` (YYYY-MM-DD). "
            "If only `start_date` is given, you must provide a single date search and you don't need `end_date` or `services`. "
            "If `end_date` is provided, it will perform a date range search. "
            "If a date range search, you must also provide an `end_date` (YYYY-MM-DD) and a `services` list (max 6). "
            "Valid services are: " + ", ".join(SERVICE_MAPPING.keys()) + ". "
            "This tool requires manual CAPTCHA input in the terminal."
        ),
        args_schema=ShippingIndicesInput,
        coroutine=scfi_scraper.scrape_shipping_data
    )

# Inisialisasi tool
# shipping_tool = get_shipping_scfi_indices_tool()

# print("--- Example 1: Single Date Search ---")
# # LLM akan memanggil ini jika ditanya: "What were the shipping indices on July 4, 2025?"
# single_date_result = shipping_tool.run({"start_date": "2025-07-09"})
# print(single_date_result)

# print("--- Example 2: Multiple Services and Date Range Search ---")
# LLM akan memanggil ini jika ditanya: "Show me the USEC and Mediterranean indices from April 9 to July 9, 2025"
# multiple_search_result = shipping_tool.run({
#     "start_date": "2025-04-09",
#     "end_date": "2025-07-09",
#     "services": ["USEC (Base port)", "Mediterranean (Base port)", "Southeast Asia (Singapore)"]
# })
# print(multiple_search_result)

In [7]:
import asyncio # ❗️ Pastikan mengimpor asyncio
from langchain_google_vertexai import ChatVertexAI
from langchain.agents import initialize_agent, AgentType
# from shipping_data_tool import get_shipping_indices_tool

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "sinarmas-app-price-analytics.json"

async def main():
    """Fungsi utama untuk menjalankan LangChain Agent dengan tool kustom."""
    print("🚀 Initializing a LangChain Agent with Vertex AI (Gemini)...")

    # Inisialisasi LLM menggunakan ChatVertexAI
    llm = ChatVertexAI(
        model_name="gemini-2.5-flash",
        project="sinarmas-app-price-analytics",
        location="us-central1",
        temperature=0.7,
        top_p=0.8
    )

    # Siapkan daftar tools yang akan digunakan oleh agent
    tools = [
        get_shipping_scfi_indices_tool(),
    ]
    print(f"✅ Tools loaded: {[tool.name for tool in tools]}")

    # Inisialisasi Agent menggunakan StructuredChatAgent
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # Gunakan StructuredChatAgent
        verbose=True,  # verbose=True akan menampilkan "pikiran" agent saat bekerja
        handle_parsing_errors=True  # Penting untuk menangani jika output LLM tidak sempurna
    )

    # Jalankan Agent dengan sebuah prompt
    print("\n" + "=" * 50)
    print("🤖 Agent is ready. Sending a prompt...")
    prompt = "Tunjukkan pergerakan harga untuk USEC, Mediterranean, Singapore di tanggal 3 April sampai 14 Juli 2025"

    # Agent akan menganalisis prompt dan menjalankan tool dengan argumen yang tepat
    result = await agent.arun(prompt)
    print("\n" + "--- FINAL ANSWER ---")
    print(result)

if __name__ == "__main__":
    try:
        asyncio.run(main())
    except RuntimeError as e:
        if "cannot be called from a running event loop" in str(e):
            import nest_asyncio
            nest_asyncio.apply()
            asyncio.run(main())
        else:
            raise e

🚀 Initializing a LangChain Agent with Vertex AI (Gemini)...
✅ Tools loaded: ['shanghai_freight_indices_scraper']

🤖 Agent is ready. Sending a prompt...


> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "shanghai_freight_indices_scraper",
  "action_input": {
    "start_date": "2025-04-03",
    "end_date": "2025-07-14",
    "services": [
      "USEC (Base port)",
      "Mediterranean (Base port)",
      "Southeast Asia (Singapore)"
    ]
  }
}
```

I0000 00:00:1752646799.149490 6612320 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


✅ Detected logged-in username: 
🔐 Logging in...
Captcha output: 3346
Captcha tool result: 3346
✅ Login successful!
🔎 Multiple date and service search...
Table HTML extracted successfully!
✅ Table parsed and converted to JSON.

Observation: [
  {
    "0":"Shanghai Containerized Freight Index",
    "1":"Shanghai Containerized Freight Index",
    "2":"Shanghai Containerized Freight Index",
    "3":"Shanghai Containerized Freight Index",
    "4":"Shanghai Containerized Freight Index",
    "5":"Shanghai Containerized Freight Index",
    "6":"Shanghai Containerized Freight Index",
    "7":"Shanghai Containerized Freight Index"
  },
  {
    "0":"TIME  (YYYY-MM-DD)",
    "1":"Comprehensive Index",
    "2":"Mediterranean (Base port)",
    "3":"USEC (Base port)",
    "4":"Southeast Asia (Singapore)",
    "5":null,
    "6":null,
    "7":null
  },
  {
    "0":"2025-04-03",
    "1":"1392.78",
    "2":"2027.62",
    "3":"3306.04",
    "4":"444.72",
    "5":null,
    "6":null,
    "7":null
  },
  {
 

## Ringkasan Eksekutif

Pasar logistik dan pengiriman global pada Juni-Juli 2025 menunjukkan volatilitas yang signifikan dan tantangan operasional yang berkelanjutan. Tarif pengiriman menunjukkan tren campuran, dengan penurunan pada rute utama Asia-Eropa namun peningkatan pada rute intra-Asia. Tren harga bahan bakar relatif stabil dengan fluktuasi minor. Tantangan operasional, termasuk kemacetan pelabuhan yang meluas, kekurangan tenaga kerja, dan gangguan layanan, terus memengaruhi efisiensi rantai pasok. Secara geopolitik, situasi di Laut Merah tetap menjadi perhatian utama, menyebabkan pengalihan rute yang mahal dan peningkatan premi risiko, sementara kebijakan perdagangan AS-Tiongkok terus memengaruhi pola pemesanan.

---

## 1. Tarif Pengiriman

Tarif pengiriman menunjukkan pergerakan yang bervariasi di berbagai rute dan indeks:

*   **Drewry Freight Rate Index (Juni 2025):**
    *   Rute Singapura ke Shanghai:
        *   Kontainer 20 kaki: $400, menunjukkan penurunan 4% dari Mei 2025.
        *   Kontainer 40 kaki/40 kaki HC: $579, menunjukkan penurunan 6% dari bulan sebelumnya.
    *   *Sumber: Drewry Freight Rate Index, Juni 2025*
*   **SCFI (Shanghai Containerized Freight Index) - 4 Juli 2025:**
    *   Untuk Asia Tenggara (Singapura): Berada di 453 USD/TEU, penurunan marjinal 3 poin dari minggu sebelumnya.
    *   *Sumber: SCFI, 2025-07-04*
*   **Drewry's Intra-Asia Container Index (IACI) - Awal Juni:**
    *   Mengalami peningkatan 8%, mencapai $707 per kontainer 40 kaki.
    *   Namun, indeks ini tetap 27% lebih rendah secara tahunan (year-on-year).
    *   *Sumber: Drewry*

---

## 2. Tren Bahan Bakar

Tren harga bahan bakar menunjukkan stabilitas relatif dengan beberapa pergerakan harga yang menarik:

*   **VLSFO (Very Low Sulphur Fuel Oil):**
    *   Mengalami fluktuasi harian minor pada rata-rata global.
    *   Harga rata-rata: $590.50/MT.
    *   *Sumber: Market sources / MABUX*
*   **Perbedaan Harga 380 HSFO (High Sulphur Fuel Oil) dan VLSFO:**
    *   Selisih harga menyempit sebesar $3.00 menjadi $67.00.
    *   Nilai rata-rata mingguan di pelabuhan Singapura meningkat sebesar $4.17.
    *   *Sumber: Market sources / MABUX*

---

## 3. Pembaruan Operasional

Beberapa pelabuhan dan wilayah mengalami gangguan operasional signifikan:

*   **Kemacetan Pelabuhan:**
    *   Rotterdam, Ningbo-Zhoushan, dan Cape Town: Mengalami kemacetan pelabuhan yang menyebabkan penundaan signifikan, dengan waktu tunggu hingga 10 hari.
    *   *Sumber: Market observation*
*   **Kapasitas Lapangan (Yard Space):**
    *   Singapura: Kapasitas lapangan mendekati batasnya.
    *   *Sumber: Market observation*
*   **Tekanan Pelabuhan Tiongkok:**
    *   Pelabuhan-pelabuhan Tiongkok: Mengalami tekanan akibat lonjakan ekspor menjelang potensi perubahan tarif.
    *   *Sumber: Market observation*
*   **Gangguan Layanan Kereta Api:**
    *   Pelabuhan Hamburg: Mengalami gangguan sementara pada layanan kereta api dari 4-8 Juli 2025, memengaruhi pengiriman dan pengambilan kontainer.
    *   *Sumber: Hapag-Lloyd*
*   **Kekurangan Tenaga Kerja:**
    *   Terminal Delta II Rotterdam: Mengalami kekurangan tenaga kerja yang berkelanjutan.
    *   *Sumber: Hapag-Lloyd / Market observation*
*   **Mogok Nasional:**
    *   Antwerp: Mengalami mogok nasional pada 25 Juni 2025, menyebabkan penundaan kapal dan tongkang.
    *   *Sumber: Hapag-Lloyd*
*   **Sengketa Hukum:**
    *   Genoa: Terjadi sengketa hukum mengenai konsesi terminal yang berakhir pada akhir Juni 2025.
    *   *Sumber: Hapag-Lloyd*

---

## 4. Dampak Geopolitik

Peristiwa geopolitik terus membentuk lanskap logistik global:

*   **Laut Merah:**
    *   Serangan Houthi berlanjut pada 6 dan 7 Juli 2025, termasuk penenggelaman satu kapal dan serangan terhadap kapal lainnya.
    *   Menyebabkan pengalihan rute di sekitar Tanjung Harapan, menambah 10-14 hari waktu transit dan meningkatkan biaya hingga 300%.
    *   Premi risiko perang melonjak 167%.
    *   Transit Terusan Suez anjlok 57,5% sejak tahun 2023.
    *   *Sumber: Market analysis / News reports*
*   **Perdagangan AS-Tiongkok:**
    *   Kebijakan perdagangan yang berfluktuasi, termasuk pengurangan tarif sementara, menyebabkan pergeseran dalam pemesanan kontainer dan penempatan kapasitas.
    *   *Sumber: Market analysis*
*   **Selat Hormuz:**
    *   Ketegangan antara Israel dan Iran belum berdampak signifikan pada lalu lintas, yang tetap stabil karena peningkatan kehadiran angkatan laut.
    *   *Sumber: Market analysis*

---

## Ringkasan Utama Data

| Indeks/Item | Sumber | Nilai/Status Terbaru | Perubahan/Deskripsi |
|---|---|---|---|
| Drewry Freight Rate (SG-SH, 20ft) | Drewry Freight Rate Index, Juni 2025 | $400 | Turun 4% dari Mei 2025 |
| Drewry Freight Rate (SG-SH, 40ft/HC) | Drewry Freight Rate Index, Juni 2025 | $579 | Turun 6% dari bulan sebelumnya |
| SCFI (Asia Tenggara) | SCFI, 2025-07-04 | 453 USD/TEU | Turun 3 poin dari minggu sebelumnya |
| Drewry IACI (40ft) | Drewry | $707 | Naik 8% (awal Juni), Turun 27% YOY |
| VLSFO (global average) | Market sources / MABUX | $590.50/MT | Fluktuasi harian minor |
| Selisih Harga 380 HSFO & VLSFO | Market sources / MABUX | $67.00 | Menyempit $3.00 |
| Kemacetan Pelabuhan | Market observation | Rotterdam, Ningbo-Zhoushan, Cape Town | Penundaan hingga 10 hari |
| Laut Merah - Rerouting | Market analysis / News reports | Via Cape of Good Hope | Tambahan 10-14 hari, biaya naik hingga 300% |
| Laut Merah - Premi Risiko Perang | Market analysis / News reports | Naik 167% | |
| Transit Terusan Suez | Market analysis / News reports | Turun 57.5% | Sejak 2023 |

In [6]:
import pdfplumber

def print_pdf_text(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            text = page.extract_text()
            print(f"\n\n=== Page {i} ===\n")
            print(text if text else "[No text found]")

# Contoh penggunaan
print_pdf_text("Shanghai-Shipping-Exchange.pdf")



=== Page 1 ===

7/9/25, 10:22 PM Shanghai Shipping Exchange
Adobe Flash Player is no longer supported
Home About us Freight Indices Ship Trading Freight Rate Filing Member Services
Freight Indices
Introduction
User:
About Indices
Please input the period: (YYYY-MM-DD) 2025-07-09 Username: APPCN
FAQ
Contract Template
Indices
CCFI Shanghai Containerized Freight Index Contract Template
SSCCFFII
Previous Index Current Index Compare With
Description Unit Weighting
SCFIS 2025-06-27 2025-07-04 Last Week
CBFI Comprehensive Index 1861.51 1763.49 -98.02
CBCFI Europe (Base port) USD/TEU 20.0% 2030 2101 72
Mediterranean (Base port) USD/TEU 10.0% 2985 2869 -116
CBOFI
USWC (Base port) USD/FEU 20.0% 2578 2089 -490
CBGFI
USEC (Base port) USD/FEU 7.5% 4717 4124 -593
CDFI
Persian Gulf and Red Sea (Dubai) USD/TEU 7.5% 2060 1916 -144
FDI Australia/New Zealand (Melbourne) USD/TEU 5.0% 836 853 18
CTFI East/West Africa (Lagos) USD/TEU 2.5% 4526 4500 -26
South Africa (Durban) USD/TEU 2.5% 2641 2653 13
CSI
So

In [ ]:
page.locator("#SelLine").check()
page.get_by_role("cell", name="L1 Europe (Base port)").get_by_role("checkbox").check()
page.get_by_role("cell", name="L3 USWC (Base port)").get_by_role("checkbox").check()
page.get_by_role("cell", name="L5 Persian Gulf and Red Sea (").get_by_role("checkbox").check()
page.get_by_role("cell", name="L7 East/West Africa (Lagos)").get_by_role("checkbox").check()
page.get_by_role("cell", name="L9 South America (Santos)").get_by_role("checkbox").check()
page.get_by_role("cell", name="L11 East Japan (Base port)").get_by_role("checkbox").check()
page.get_by_role("cell", name="L13 Korea (Pusan)").get_by_role("checkbox").check()
page.get_by_role("cell", name="L15 Hong Kong (Hong Kong)").get_by_role("checkbox").check()
page.get_by_role("cell", name="L26 East Africa(Mombasa)").get_by_role("checkbox").check()
page.get_by_role("cell", name="L2 Mediterranean (Base port)").get_by_role("checkbox").check()
page.get_by_role("cell", name="L4 USEC (Base port)").get_by_role("checkbox").check()
page.get_by_role("cell", name="L6 Austrlian/New Zealand (").get_by_role("checkbox").check()
page.get_by_role("cell", name="L8 South Africa (Duban)").get_by_role("checkbox").check()
page.get_by_role("cell", name="L10 West Japan (Base port)").get_by_role("checkbox").check()
page.get_by_role("cell", name="L12 Southeast Asia (Singapore)").get_by_role("checkbox").check()
page.get_by_role("cell", name="L14 Taiwan (Kaohsiung)").get_by_role("checkbox").check()
page.get_by_role("cell", name="L25 Central/South America").get_by_role("checkbox").check()


SERVICE_MAPPING = {
    "Comprehensive Index": "#SelLine",
    "Europe (Base port)": "L1 Europe (Base port)",
    "USWC (Base port)": "L3 USWC (Base port)",
    "Persian Gulf and Red Sea": "L5 Persian Gulf and Red Sea (",
    "East/West Africa (Lagos)": "L7 East/West Africa (Lagos)",
    "South America (Santos)": "L9 South America (Santos)",
    "East Japan (Base port)": "L11 East Japan (Base port)",
    "Korea (Pusan)": "L13 Korea (Pusan)",
    "Hong Kong (Hong Kong)": "L15 Hong Kong (Hong Kong)",
    "East Africa(Mombasa)": "L26 East Africa(Mombasa)",
    "Mediterranean (Base port)": "L2 Mediterranean (Base port)",
    "USEC (Base port)": "L4 USEC (Base port)",
    "Austrlian/New Zealand (Melbourne)": "L6 Austrlian/New Zealand (",
    "South Africa (Duban)": "L8 South Africa (Duban)",
    "West Japan (Base port)": "L10 West Japan (Base port)",
    "Southeast Asia (Singapore)": "L12 Southeast Asia (Singapore)",
    "Taiwan (Kaohsiung)": "L14 Taiwan (Kaohsiung)",
    "Central/South America": "L25 Central/South America"
}

service_id = SERVICE_MAPPING.get("Southeast Asia (Singapore)")
if service_id:
    print(f"Service ID for 'Siangpor': {service_id}")
else:   
    print("Service 'Siangpor' not found in mapping.")

Service ID for 'Siangpor': L12 Southeast Asia (Singapore)
